In [6]:
import geopandas as gp
import pandas as pd
from shapely.geometry import Polygon
from routingpy.routers import MapboxOSRM
import numpy as np

mb = MapboxOSRM(api_key = "pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw")

In [7]:
dallas_libraries = gp.read_file("https://egis.dallascityhall.com/resources/Downloads/ShpZip/Library/Libraries.zip")

dallas_libraries.explore()

In [8]:
def mb_isochrone(gdf, time = [5, 10, 15], profile = "driving"):

    # Grab X and Y values in 4326
    gdf['LON_VALUE'] = gdf.to_crs(4326).geometry.x
    gdf['LAT_VALUE'] = gdf.to_crs(4326).geometry.y

    coordinates = gdf[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    # Build a list of shapes
    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    # Use minutes as input, but the API requires seconds
    time_seconds = [60 * x for x in time]

    # Given the way that routingpy works, we need to iterate through the list of 
    # coordinate pairs, then iterate through the object returned and extract the 
    # isochrone geometries.  
    for c in coordinates:
        iso_request = mb.isochrones(locations = c, profile = profile,
                                    intervals = time_seconds, polygons = "true")

        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # Here, we re-build the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # We'll need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gp.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # We are sorting the dataframe in descending order of time to improve visualization
    # (the smallest isochrones should go on top, which means they are plotted last)
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)

In [14]:
dallas_libraries.head()

,Library,Address,Telephone,Url,BranchClas,geometry,LON_VALUE,LAT_VALUE
0,Arcadia Park,1302 N Justin Ave,214-670-6446,http://www.dallaslibrary2.org/branch/arcadia.php,Branch,POINT (2457410.114 6962825.079),-96.909468,32.759149
1,Audelia Road,10045 Audelia Rd,214-670-1350,http://www.dallaslibrary2.org/branch/audelia.php,Branch,POINT (2515388.236 7009805.942),-96.718295,32.885712
2,Bachman Lake,9480 Webb Chapel Rd,214-670-6376,http://www.dallaslibrary2.org/branch/bachman.php,Branch,POINT (2472278.638 7000779.757),-96.859177,32.862835
3,Bookmarks at NorthPark Center,8687 North Central Expwy,214-671-1381,http://www.dallaslibrary2.org/branch/bookmarks...,Branch,POINT (2498263.282 7003408.784),-96.774418,32.868916
4,Dallas West,2332 Singleton Blvd,214-670-6445,http://www.dallaslibrary2.org/branch/dallaswes...,Branch,POINT (2472480.223 6970036.566),-96.860083,32.778332


In [9]:
library_isos = mb_isochrone(dallas_libraries, time = 5, 
                            profile = "driving-traffic")

library_isos.explore()

In [15]:
library_isos.head()

,Library,Address,Telephone,Url,BranchClas,geometry,time
0,Arcadia Park,1302 N Justin Ave,214-670-6446,http://www.dallaslibrary2.org/branch/arcadia.php,Branch,"POLYGON ((-96.90947 32.75998, -96.91245 32.760...",5
1,Audelia Road,10045 Audelia Rd,214-670-1350,http://www.dallaslibrary2.org/branch/audelia.php,Branch,"POLYGON ((-96.7173 32.90172, -96.71957 32.9017...",5
2,Bachman Lake,9480 Webb Chapel Rd,214-670-6376,http://www.dallaslibrary2.org/branch/bachman.php,Branch,"POLYGON ((-96.86618 32.88605, -96.86845 32.885...",5
3,Bookmarks at NorthPark Center,8687 North Central Expwy,214-671-1381,http://www.dallaslibrary2.org/branch/bookmarks...,Branch,"POLYGON ((-96.76942 32.89364, -96.76962 32.888...",5
4,Dallas West,2332 Singleton Blvd,214-670-6445,http://www.dallaslibrary2.org/branch/dallaswes...,Branch,"POLYGON ((-96.85208 32.80684, -96.85339 32.805...",5


In [17]:
library_isos.dtypes

Library         object
Address         object
Telephone       object
Url             object
BranchClas      object
geometry      geometry
time             int64
dtype: object

In [19]:
dickies = gp.tools.geocode("1911 Montgomery St, Fort Worth, TX 76107")

dickies_isos = mb_isochrone(dickies, time = [5, 10, 15], profile = "driving-traffic")

dickies_isos.explore(column = "time")